In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import



In [ ]:
import itertools
import os
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

print(tf.__version__)

2.15.0


In [ ]:
df = pd.read_csv('/content/consumer_complaints.csv')
df.head()

<ipython-input-7-ee38e8fda86f>:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/consumer_complaints.csv')


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074.0
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080.0
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473.0
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326.0
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067.0


In [ ]:

col = ['consumer_complaint_narrative', 'product']
df = df[col]


df = df[pd.notnull(df['consumer_complaint_narrative'])]
df.head()

,consumer_complaint_narrative,product
190126,XXXX has claimed I owe them {$27.00} for XXXX ...,Debt collection
190135,Due to inconsistencies in the amount owed that...,Consumer Loan
190155,In XX/XX/XXXX my wages that I earned at my job...,Mortgage
190207,I have an open and current mortgage with Chase...,Mortgage
190208,XXXX was submitted XX/XX/XXXX. At the time I s...,Mortgage


In [ ]:
df.isnull().sum()

,0
consumer_complaint_narrative,0
product,0


In [ ]:
df['product'].value_counts()

,count
product,
Debt collection,2158
Mortgage,1703
Credit reporting,1126
Credit card,832
Bank account or service,598
Consumer Loan,393
Student loan,318
Payday loan,91
Money transfers,82


In [ ]:
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

Train size: 5887
Test size: 1472


In [ ]:
train_narrative = df['consumer_complaint_narrative'][:train_size]
train_product = df['product'][:train_size]

test_narrative = df['consumer_complaint_narrative'][train_size:]
test_product = df['product'][train_size:]

In [ ]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

tokenize.fit_on_texts(train_narrative) # only fit on train
x_train = tokenize.texts_to_matrix(train_narrative)
x_test = tokenize.texts_to_matrix(test_narrative)

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_product)
y_train = encoder.transform(train_product)
y_test = encoder.transform(test_product)

In [ ]:
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [ ]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)


x_train shape: (5887, 1000)
x_test shape: (1472, 1000)
y_train shape: (5887, 11)
y_test shape: (1472, 11)


In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
batch_size = 32
epochs = 5

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
166/166 [==============================] - 5s 14ms/step - loss: 1.1517 - accuracy: 0.6506 - val_loss: 0.8171 - val_accuracy: 0.7589
Epoch 2/5
166/166 [==============================] - 2s 15ms/step - loss: 0.5855 - accuracy: 0.8292 - val_loss: 0.6893 - val_accuracy: 0.7946
Epoch 3/5
166/166 [==============================] - 2s 15ms/step - loss: 0.4149 - accuracy: 0.8722 - val_loss: 0.6813 - val_accuracy: 0.7980
Epoch 4/5
166/166 [==============================] - 2s 10ms/step - loss: 0.3005 - accuracy: 0.9086 - val_loss: 0.6978 - val_accuracy: 0.7861
Epoch 5/5
166/166 [==============================] - 2s 9ms/step - loss: 0.2183 - accuracy: 0.9370 - val_loss: 0.6961 - val_accuracy: 0.7895


In [ ]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])
print('Test score:',score[0])

46/46 [==============================] - 1s 14ms/step - loss: 0.7181 - accuracy: 0.7962
Test accuracy: 0.7961956262588501
Test score: 0.7180718779563904


In [ ]:
text_labels = encoder.classes_

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_narrative.iloc[i][:50], "...")
    print('Actual label:' + test_product.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

1/1 [==============================] - 0s 108ms/step
Fell upon hard financial times! XXXX business went ...
Actual label:Consumer Loan
Predicted label: Debt collection

1/1 [==============================] - 0s 21ms/step
i bought a motor vehicle from drive time in Florid ...
Actual label:Consumer Loan
Predicted label: Consumer Loan

1/1 [==============================] - 0s 20ms/step
OK back in XXXX XXXX i purchased a XXXX XXXX from  ...
Actual label:Consumer Loan
Predicted label: Credit card

1/1 [==============================] - 0s 32ms/step
The company claims that my husband owes them {$840 ...
Actual label:Debt collection
Predicted label: Debt collection

1/1 [==============================] - 0s 20ms/step
I was being charge with fees from an Appt complex  ...
Actual label:Credit reporting
Predicted label: Credit reporting

1/1 [==============================] - 0s 24ms/step
XXXX unrecognized charges classified as Bank Fees  ...
Actual label:Credit card
Predicted label: Credit car